### Notebook for analyzing BioCentury Research Farm East Bilsland field site - author @ Matt Nowatzke

In [2]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import database as db
import gdal
import rasterio as rio
import run_apsim
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from rasterstats import zonal_stats
from munging import get_centroid
from apsim.daymet import create_excel_met
from apsim.apsim_input_writer import create_mukey_runs
from apsim.apsim_output_parser import parse_all_output_field, parse_summary_output_field

### Load management data for 2018 soybeans following corn and 2019 corn following soybeans.

In [ ]:
ebilsland_corn_mgmt_2018 = json.loads( open( 'crop_jsons/ebilsland_cfs_2018.json', 'r' ).read() )
ebilsland_soy_mgmt_2019 = json.loads( open( 'crop_jsons/ebilsland_sfc_2019.json', 'r' ).read() )

In [ ]:
ebilsland_soy_mgmt_2019

### Create database connection and get a list of all the unique soils (mukeys) for the field.  
Each soil mapunit has unique musym identifier and a mapunit key (mukey) which relates to soil properties.

In [3]:
dbconn = db.connect_to_db('database.ini')

In [ ]:
ebilsland_ssurgo = gpd.read_file("C:/Users/mjn/Documents/Foresite/yield_monitor_data/analyses/ebilsland/ebilsland_ssurgo_2019.geojson")
ebilsland_mukeys = list(np.unique(ebilsland_ssurgo['mukey']))
ebilsland_mukeys

### Get the centroid of the field and create met file.
This met file--if created on a Windows machine--is a an Excel file that should be saved afterwards as a Formatted Text file (*.prn)  
See: https://www.apsim.info/support/apsim-training-manuals/creating-an-apsim-met-file-using-excel/  
This is a known problem with Pandas on Windows. If creating the met file on Mac or Linux, feel free to use the daymet.create_met function instead.

In [ ]:
ebilsland_centroid = get_centroid(ebilsland_ssurgo, 'areasymbol', 'geometry')
create_excel_met(ebilsland_centroid[0], ebilsland_centroid[1], 2012, 2019, 'Ebilsland')
print(f'East Bilsland centroid located at {ebilsland_centroid}')

### Create runs using the default soil calcs or Saxton Rawls

In [ ]:
create_mukey_runs(ebilsland_mukeys, dbconn, 'sfc', 'Ebilsland.met', 'EbilslandDefault', start_year=2017, end_year=2019, sfc_mgmt=ebilsland_soy_mgmt_2019, cfs_mgmt=ebilsland_corn_mgmt_2018)
create_mukey_runs(ebilsland_mukeys, dbconn, 'sfc', 'Ebilsland.met', 'EbilslandSaxton', start_year=2017, end_year=2019, sfc_mgmt=ebilsland_soy_mgmt_2019, cfs_mgmt=ebilsland_corn_mgmt_2018, saxton=True)

### Run simulations

In [ ]:
run_apsim.run_all_simulations(apsim_files_path="apsim_files\\EbilslandSaxton\\*.apsim", sim_files_path="apsim_files\\EbilslandSaxton\\*.sim")

In [ ]:
run_apsim.run_all_simulations(apsim_files_path="apsim_files\\EbilslandDefault\\*.apsim", sim_files_path="apsim_files\\EbilslandDefault\\*.sim")

### View output from APSIM runs for default soil profiles and saxton soil profiles.

In [ ]:
ebilsland_soy_output_2019_saxton = parse_summary_output_field("apsim_files/EbilslandSaxton/", year=2019)
ebilsland_soy_output_2019_saxton

In [ ]:
ebilsland_soy_output_2019_default = parse_summary_output_field("apsim_files/EbilslandDefault/", year=2019)
ebilsland_soy_output_2019_default

Get zonal stats from yield monitor data to compare to simulated yields

In [4]:
ebilsland_raster = "C:\\Users\\mjn\\Documents\\Foresite\\yield_monitor_data\\analyses\\ebilsland\\ebilsland_soy_2019_raster_30.tif"
ebilsland_ssurgo_geojson = 'C:/Users/mjn/Documents/Foresite/yield_monitor_data/analyses/ebilsland/ebilsland_ssurgo_2019.geojson'

In [30]:
stats = zonal_stats(ebilsland_ssurgo_geojson, ebilsland_raster, geojson_out=True, stats=['min', 'max', 'median', 'mean', 'std', 'range'])

In [7]:
ebilsland_soy_2019_stats = gpd.GeoDataFrame.from_features(stats)

In [8]:
ebilsland_soy_2019_stats

,geometry,fid,Field,Obj__Id,Bnd_Name,objectid,areasymbol,spatialver,musym,mukey,shape_length,shape_area,min,max,mean,std,median,range
0,"MULTIPOLYGON (((-93.79891 41.90873, -93.79898 ...",1,East Bilsland,1.0,East Bilsland,13447276,IA015,10.0,6,2550285,400.644320,7114.270891,15.034800,68.904800,57.189427,14.771053,60.132149,53.870001
1,"MULTIPOLYGON (((-93.80268 41.90737, -93.80269 ...",2,East Bilsland,1.0,East Bilsland,13449079,IA015,10.0,L107,2835012,523.917874,3154.015393,60.053600,65.421501,62.475149,1.728630,62.727150,5.367901
2,"MULTIPOLYGON (((-93.80271 41.91005, -93.80273 ...",3,East Bilsland,1.0,East Bilsland,13455246,IA015,10.0,L138B,2765522,349.123249,1190.190208,20.321899,28.609301,24.465599,4.143701,24.465599,8.287401
3,"MULTIPOLYGON (((-93.79608 41.90707, -93.79712 ...",4,East Bilsland,1.0,East Bilsland,13455735,IA015,10.0,L138B,2765522,1484.287985,26235.367905,15.305000,69.767303,54.350890,13.771772,59.078602,54.462303
4,"MULTIPOLYGON (((-93.80013 41.90708, -93.80020 ...",5,East Bilsland,1.0,East Bilsland,13458179,IA015,10.0,L138B,2765522,8373.176363,40329.439255,15.566300,71.133904,55.358508,11.611535,59.536148,55.567603
5,"MULTIPOLYGON (((-93.80182 41.91232, -93.80183 ...",6,East Bilsland,1.0,East Bilsland,13458216,IA015,10.0,L138B,2765522,383.083178,6525.337137,16.549999,58.070099,35.492076,15.213239,33.372849,41.520100
6,"MULTIPOLYGON (((-93.79722 41.91455, -93.79583 ...",7,East Bilsland,1.0,East Bilsland,13458451,IA015,10.0,L138B,2765522,1702.878158,7290.170439,15.357500,96.371201,52.391959,23.342302,54.447548,81.013700
7,"MULTIPOLYGON (((-93.79568 41.90998, -93.79569 ...",8,East Bilsland,1.0,East Bilsland,13461421,IA015,10.0,L138B,2765522,2247.863067,78396.541986,15.285800,78.290497,60.465893,8.377456,62.115501,63.004697
8,"MULTIPOLYGON (((-93.80178 41.91389, -93.80179 ...",9,East Bilsland,1.0,East Bilsland,13461925,IA015,10.0,L138B,2765522,675.417138,2582.220504,18.304600,67.944099,39.065453,18.427182,35.006550,49.639500
9,"MULTIPOLYGON (((-93.79759 41.91050, -93.79773 ...",10,East Bilsland,1.0,East Bilsland,13466907,IA015,10.0,L507,2922007,724.117501,15066.090284,43.532799,74.706596,62.121024,5.835248,63.057098,31.173798


In [19]:
def wkb_hexer(line):
    return line.wkb_hex

In [46]:
ebilsland_soy_ym = ebilsland_soy_2019_stats[['geometry', 'Field', 'objectid', 'areasymbol', 'musym', 'mukey', 'shape_area', 'mean', 'median', 'min', 'max', 'std', 'range']]
ebilsland_soy_ym.sort_values('mukey')

,geometry,Field,objectid,areasymbol,musym,mukey,shape_area,mean,median,min,max,std,range
0,"MULTIPOLYGON (((-93.79891 41.90873, -93.79898 ...",East Bilsland,13447276,IA015,6,2550285,7114.270891,57.189427,60.132149,15.034800,68.904800,14.771053,53.870001
2,"MULTIPOLYGON (((-93.80271 41.91005, -93.80273 ...",East Bilsland,13455246,IA015,L138B,2765522,1190.190208,24.465599,24.465599,20.321899,28.609301,4.143701,8.287401
3,"MULTIPOLYGON (((-93.79608 41.90707, -93.79712 ...",East Bilsland,13455735,IA015,L138B,2765522,26235.367905,54.350890,59.078602,15.305000,69.767303,13.771772,54.462303
4,"MULTIPOLYGON (((-93.80013 41.90708, -93.80020 ...",East Bilsland,13458179,IA015,L138B,2765522,40329.439255,55.358508,59.536148,15.566300,71.133904,11.611535,55.567603
5,"MULTIPOLYGON (((-93.80182 41.91232, -93.80183 ...",East Bilsland,13458216,IA015,L138B,2765522,6525.337137,35.492076,33.372849,16.549999,58.070099,15.213239,41.520100
6,"MULTIPOLYGON (((-93.79722 41.91455, -93.79583 ...",East Bilsland,13458451,IA015,L138B,2765522,7290.170439,52.391959,54.447548,15.357500,96.371201,23.342302,81.013700
7,"MULTIPOLYGON (((-93.79568 41.90998, -93.79569 ...",East Bilsland,13461421,IA015,L138B,2765522,78396.541986,60.465893,62.115501,15.285800,78.290497,8.377456,63.004697
8,"MULTIPOLYGON (((-93.80178 41.91389, -93.80179 ...",East Bilsland,13461925,IA015,L138B,2765522,2582.220504,39.065453,35.006550,18.304600,67.944099,18.427182,49.639500
11,"MULTIPOLYGON (((-93.80269 41.90877, -93.80268 ...",East Bilsland,13470727,IA015,L55,2834849,23762.476705,48.127396,55.365799,15.333300,73.863197,19.131288,58.529898
12,"MULTIPOLYGON (((-93.79712 41.90707, -93.79801 ...",East Bilsland,13471951,IA015,L55,2834849,6253.694122,46.154128,50.864498,16.247299,63.756901,16.312921,47.509602


In [47]:
ebilsland_soy_ym = ebilsland_soy_ym.round({'mean':2,'median':2, 'min':2, 'max':2, 'std':2, 'range':2})

In [48]:
ebilsland_soy_ym

,geometry,Field,objectid,areasymbol,musym,mukey,shape_area,mean,median,min,max,std,range
0,"MULTIPOLYGON (((-93.79891 41.90873, -93.79898 ...",East Bilsland,13447276,IA015,6,2550285,7114.270891,57.19,60.13,15.03,68.90,14.77,53.87
1,"MULTIPOLYGON (((-93.80268 41.90737, -93.80269 ...",East Bilsland,13449079,IA015,L107,2835012,3154.015393,62.48,62.73,60.05,65.42,1.73,5.37
2,"MULTIPOLYGON (((-93.80271 41.91005, -93.80273 ...",East Bilsland,13455246,IA015,L138B,2765522,1190.190208,24.47,24.47,20.32,28.61,4.14,8.29
3,"MULTIPOLYGON (((-93.79608 41.90707, -93.79712 ...",East Bilsland,13455735,IA015,L138B,2765522,26235.367905,54.35,59.08,15.31,69.77,13.77,54.46
4,"MULTIPOLYGON (((-93.80013 41.90708, -93.80020 ...",East Bilsland,13458179,IA015,L138B,2765522,40329.439255,55.36,59.54,15.57,71.13,11.61,55.57
5,"MULTIPOLYGON (((-93.80182 41.91232, -93.80183 ...",East Bilsland,13458216,IA015,L138B,2765522,6525.337137,35.49,33.37,16.55,58.07,15.21,41.52
6,"MULTIPOLYGON (((-93.79722 41.91455, -93.79583 ...",East Bilsland,13458451,IA015,L138B,2765522,7290.170439,52.39,54.45,15.36,96.37,23.34,81.01
7,"MULTIPOLYGON (((-93.79568 41.90998, -93.79569 ...",East Bilsland,13461421,IA015,L138B,2765522,78396.541986,60.47,62.12,15.29,78.29,8.38,63.00
8,"MULTIPOLYGON (((-93.80178 41.91389, -93.80179 ...",East Bilsland,13461925,IA015,L138B,2765522,2582.220504,39.07,35.01,18.30,67.94,18.43,49.64
9,"MULTIPOLYGON (((-93.79759 41.91050, -93.79773 ...",East Bilsland,13466907,IA015,L507,2922007,15066.090284,62.12,63.06,43.53,74.71,5.84,31.17


In [49]:
ebilsland_soy_ym.to_file("ebilsland_stats.geojson", driver='GeoJSON')

In [50]:
ebilsland_soy_ym['geometry'] = ebilsland_soy_ym['geometry'].apply(wkb_hexer)

In [51]:
ebilsland_pd = pd.DataFrame(ebilsland_soy_ym)

In [55]:
db_schema = 'biocent_farms'
db_table = 'ebilsland_soy_2019_yield_zonal_stats' 
ebilsland_pd.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

#### Get distribution of observed yields

In [ ]:
with rio.open(ebilsland_raster) as src:
    ebilsland_raster_hist = src.read(1, masked=True)
ebilsland_raster_hist = ebilsland_raster_hist.ravel()

In [ ]:
plt.hist(ebilsland_raster_hist,
        bins=[10,20,30,40,50,60,70,80,90,100],
        color='purple')
plt.xlabel('Soybean yield')
plt.ylabel('Frequency')
plt.title('East Bilsland 2019 Soybean Yield')
plt.show()

![SSURGO soils with raster yield overlay](images/ebilsland_ssurgo_w_rasteryield.png "East Bilsland SSURGO soils with yield overlay")  

## Get predicted vs observed  
Filter the data and left join to get df of default soil prediction vs. observed mean and Saxton soil bs observed mean.

In [ ]:
#get yield monitor data
ebilsland_soy_ym = ebilsland_soy_2019_stats[['objectid', 'mukey', 'mean']]

In [ ]:
#get predictions
default_apsim_pred = ebilsland_soy_output_2019_default[['mukey', 'soy_buac']]
saxton_apsim_pred = ebilsland_soy_output_2019_saxton[['mukey', 'soy_buac']]

In [ ]:
#merge to compare
pred_vs_obs_default = ebilsland_soy_ym.merge(default_apsim_pred, on='mukey', how='left')
pred_vs_obs_default.columns = ['objectid', 'mukey', 'observed', 'predicted']
pred_vs_obs_default['resid'] = pred_vs_obs_default['predicted'] - pred_vs_obs_default['observed']
pred_vs_obs_saxton = ebilsland_soy_ym.merge(saxton_apsim_pred, on='mukey', how='left')
pred_vs_obs_saxton.columns = ['objectid', 'mukey', 'observed', 'predicted']
pred_vs_obs_saxton['resid'] = pred_vs_obs_saxton['predicted'] - pred_vs_obs_saxton['observed']

In [ ]:
pred_vs_obs_default

In [ ]:
pred_vs_obs_saxton

## Results from 'default' soil calculations

Round the buac and store in tuples to preserve order for comparison.

In [ ]:
yield_monitor = tuple(pred_vs_obs_default['observed'])
yield_monitor = [round(num, 2) for num in yield_monitor]
def_apsim_predicted = tuple(pred_vs_obs_default['predicted'])
def_apsim_predicted = [round(num, 2) for num in def_apsim_predicted]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(yield_monitor, def_apsim_predicted)
ax.plot([30,80],[30,80])
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
sns.set(style="whitegrid")
sns.residplot(def_apsim_predicted, yield_monitor)

In [ ]:
rmse = mean_squared_error(yield_monitor, def_apsim_predicted, squared=False)
rmse

In [ ]:
r_sq = r2_score(yield_monitor, def_apsim_predicted)
r_sq

In [ ]:
corr_matrix = np.corrcoef(yield_monitor, def_apsim_predicted)
corr_xy = corr_matrix[0,1]
r_squared = corr_xy**2
r_squared

### Saxton results

In [ ]:
yield_monitor = tuple(pred_vs_obs_saxton['observed'])
yield_monitor = [round(num, 2) for num in yield_monitor]
sax_apsim_predicted = tuple(pred_vs_obs_saxton['predicted'])
sax_apsim_predicted = [round(num, 2) for num in sax_apsim_predicted]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(yield_monitor, sax_apsim_predicted)
ax.plot([30,80],[30,80])
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
rmse = mean_squared_error(yield_monitor, sax_apsim_predicted, squared=False)
rmse

In [ ]:
r_sq = r2_score(yield_monitor, sax_apsim_predicted)
r_sq

In [ ]:
corr_matrix = np.corrcoef(yield_monitor, sax_apsim_predicted)
corr_xy = corr_matrix[0,1]
r_squared = corr_xy**2
r_squared

In [ ]:
dem_path =
dem = rio.open()